In [ ]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
import seaborn as sns
from keras import layers
from keras.applications import DenseNet121, MobileNetV2
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score, confusion_matrix
import scipy
import tensorflow as tf
from tqdm import tqdm

%matplotlib inline

Set random seed for reproducibility.

In [ ]:
np.random.seed(2019)
tf.random.set_seed(2019)

# Loading & Exploration

In [ ]:
train_df = pd.read_csv('../input/valid-and-test-ta/x_train_2.csv')
valid_df = pd.read_csv('../input/valid-and-test-ta/x_valid_2.csv')
test_df = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)
train_df.head()

In [ ]:
#resample
from sklearn.utils import resample
X=train_df
normal=X[X.diagnosis==0]
mild=X[X.diagnosis==1]
moderate=X[X.diagnosis==2]
severe=X[X.diagnosis==3]
pdr=X[X.diagnosis==4]

#downsampled
mild = resample(mild,
                replace=True, # sample with replacement
                n_samples=700, # match number in majority class
                random_state=2020) # reproducible results
moderate = resample(moderate,
                    replace=False, # sample with replacement
                    n_samples=700, # match number in majority class
                    random_state=2020) # reproducible results
severe = resample(severe,
                  replace=True, # sample with replacement
                  n_samples=700, # match number in majority class
                  random_state=2020) # reproducible results
normal = resample(normal,
                  replace=False, # sample with replacement
                  n_samples=700, # match number in majority class
                  random_state=2020) # reproducible results
pdr = resample(pdr,
               replace=True, # sample with replacement
               n_samples=700, # match number in majority class
               random_state=2020) # reproducible results    

# combine minority and downsampled majority
sampled = pd.concat([normal, mild, moderate, severe, pdr])

# checking counts
sampled.diagnosis.value_counts()

train_df = sampled
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [ ]:
#Mengecek apakah ukuran sudah sesuai
print('Number of train samples: ', train_df.shape[0])
print('Number of test samples: ', valid_df.shape[0])

train_df.head()

# Resize Images

We will resize the images to 224x224, then create a single numpy array to hold the data.

In [ ]:
def preprocess_image(image_path, desired_size=224):
    im = Image.open(image_path)
    im = im.resize((desired_size, )*2, resample=Image.LANCZOS)
    
    return im

In [ ]:
N = train_df.shape[0]
x_train = np.empty((N, 224, 224, 3), dtype=np.float32)

for i, image_id in enumerate(tqdm(train_df['id_code'])):
    x_train[i, :, :, :] = preprocess_image(
        f'../input/aptos2019-blindness-detection/train_images/{image_id}.png'
    )

In [ ]:
N = valid_df.shape[0]
x_val = np.empty((N, 224, 224, 3), dtype=np.float32)

for i, image_id in enumerate(tqdm(valid_df['id_code'])):
    x_val[i, :, :, :] = preprocess_image(
        f'../input/aptos2019-blindness-detection/train_images/{image_id}.png'
    )

In [ ]:
N = test_df.shape[0]
x_test = np.empty((N, 224, 224, 3), dtype=np.float32)

for i, image_id in enumerate(tqdm(test_df['id_code'])):
    x_test[i, :, :, :] = preprocess_image(
        f'../input/aptos2019-blindness-detection/test_images/{image_id}.png'
    )

In [ ]:
y_train = train_df['diagnosis']
y_val = valid_df['diagnosis']
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)

In [ ]:
BATCH_SIZE = 32

def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.1,  # set range for random zoom
        rotation_range = 360,
        # set mode for filling points outside the input boundaries
        fill_mode='constant',
        cval=0.,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
    )
data_generator = create_datagen().flow(x_train, y_train, batch_size=BATCH_SIZE,shuffle=False)

# Model: MobilenetV2

In [ ]:
model = load_model('../input/modell/model_2.h5')
model.summary()

In [ ]:
import keras
layer_name = 'dense_2'
intermediate_layer_model = keras.Model(inputs=model.input,
                                       outputs=model.get_layer(layer_name).output)
intermediate_layer_model.summary()

## Submit

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
y_train_pred = intermediate_layer_model.predict(x_train)
y_valid_pred = intermediate_layer_model.predict(x_val)
y_test_pred = intermediate_layer_model.predict(x_test)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(),DecisionTreeClassifier())

In [ ]:
clf.fit(y_train_pred,y_train)
y_val_pred = clf.predict(y_valid_pred)

kappa_val = cohen_kappa_score(
            valid_df['diagnosis'].astype('int'),
            y_val_pred, 
            weights='quadratic'
        )

print("kappa scores {:.2f}".format(kappa_val))

In [ ]:
labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
cnf_matrix = confusion_matrix(valid_df['diagnosis'].astype('int'), y_val_pred)
df_cm = pd.DataFrame(cnf_matrix, index=labels, columns=labels)
plt.figure(figsize=(16, 7))
sns.heatmap(df_cm, annot=True, cmap="Blues")
plt.show()

In [ ]:
y_training_pred = clf.predict(y_train_pred)
labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
cnf_matrix = confusion_matrix(train_df['diagnosis'].astype('int'), y_training_pred)
df_cm = pd.DataFrame(cnf_matrix, index=labels, columns=labels)
plt.figure(figsize=(16, 7))
sns.heatmap(df_cm, annot=True, cmap="Blues")
plt.show()

In [ ]:
y_head_lr = clf.predict(y_test_pred)

In [ ]:
test_df['diagnosis'] = y_head_lr
test_df['diagnosis'].hist()
test_df['diagnosis'].value_counts()

In [ ]:
test_df['diagnosis'] = test_df['diagnosis'].astype(int)
test_df.to_csv('submission.csv',index=False)